In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Q1. Estimating the average number of people that like an Elon Musk Tweet
Assumption(s):
* Each person is liking any given tweet on only one account, not logging into multiple and liking the same tweet multiple times

In [5]:
total_tweets = 0
total_likes = 0

for filename in filenames:
# filename = '/kaggle/input/elon-musk-tweets-2010-2021/2011.csv'
    df = pd.read_csv(os.path.join(dirname, filename), usecols=['nlikes'])
#     print("Year:", filename, "Number of Tweets in Year:",df.shape[0])
    total_tweets += df.shape[0]
    total_likes += df.sum()

average_likes = (int)(total_likes/total_tweets)
print("Total Number of Tweets:",total_tweets,"Total Likes:",total_likes,"Average Number of Likes per Tweet:",average_likes)

# Q2. Calculate average number of views for a given Elon Musk tweet
Assumption(s):
* Engagement rate, made up of the sum of likes, replies, and retweets divided by the views, is equal to 0.05

In [6]:
engagement_rate = 0.05 # (likes + replies + retweets)/views
total_tweets = 0
total_likes = 0
total_replies = 0
total_retweets = 0

for filename in filenames:
    df = pd.read_csv(os.path.join(dirname, filename), usecols=['nlikes', 'nreplies', 'nretweets'])
#     print("Year:", filename, "Number of Tweets in Year:",df.shape[0], "Number of Likes:", df.loc[:,'nlikes'].sum(), "Number of replies:",df.loc[:,'nreplies'].sum(), "Number of retweets:",df.loc[:,'nretweets'].sum())
    total_tweets += df.shape[0]
    total_likes += df.loc[:,'nlikes'].sum()
    total_replies += df.loc[:,'nreplies'].sum()
    total_retweets += df.loc[:,'nretweets'].sum()

total_views = (int)((total_likes+total_replies+total_retweets)/engagement_rate)
average_views = (int)(total_views/total_tweets)

print("Total number of views:",total_views,"Average number of views per tweet:",average_views)

# Q3. Estimating average amount of data per Elon Musk Tweet
**Maximum size of image on Twitter is 5 MB | 
Maximum size of GIF on Twitter is 5 MB |
Maximum size of video on Twitter is 1 GB**

Assumption(s):
* One Megabyte is equal to 1,000,000 [one million] bytes
* One Kilobyte is equal to 1,000 [one thousand] bytes
* Each character is 1 bytes
* Average size of tweeted video [.mov, .mp4] is 25 MB
* Average size of tweeted image [.jpg, .png] is 100 KB {Sampled random image sizes in KB: [21,23,44,88,43,16,113,127,25,92], rounded avg to nearest 100}

In [7]:
total_tweets = 0
total_media_tweets = 0
total_characters = 0
total_photos = 0
total_thumbnails = 0

for filename in filenames:
# filename = '/kaggle/input/elon-musk-tweets-2010-2021/2011.csv'
#     print('filename',filename)
    df = pd.read_csv(os.path.join(dirname, filename), usecols=['tweet','photos','video','thumbnail'])
    total_tweets += df.shape[0]
    total_characters += len(df.astype(str)['tweet'].sum())
    total_photos += df[df.astype(str)['photos'] != '[]']['photos'].count()
    total_thumbnails += df['thumbnail'].count()
#     print('thumbnail:',df['thumbnail'].count())
#     print('video',df['video'].sum())
#     print(total_photos)
#     print('photos:',df[df.astype(str)['photos'] != '[]']['photos'].count())

total_videos = total_thumbnails - total_photos

character_bytes = total_characters * 1
photo_bytes = total_photos*100*1000
video_bytes = total_videos*25*1000000

total_bytes = character_bytes+photo_bytes+video_bytes
average_bytes = (int)(total_bytes/total_tweets)
average_megabytes = average_bytes/1000000

print('Average bytes per tweet:', average_bytes)
print('Average megabytes per tweet',average_megabytes)

# print(total_tweets,total_bytes,total_videos,video_bytes,average_bytes)
# print('character',total_characters,'vids', total_videos, 'photos:',total_photos,'thumb',total_thumbnails)

# Q4. Estimate total data transfer from a given Elon Musk tweet to users
Assumption(s): 
* Twitter stores a copy of the user_id and name for every tweet sent out
* Twitter stores data for language of the tweet
* One Megabyte is equal to 1,000,000 [one million] bytes
* One Kilobyte is equal to 1,000 [one thousand] bytes
* Each character is 1 bytes
* Average size of tweeted video [.mov, .mp4] is 25 MB
* Average size of tweeted image [.jpg, .png] is 100 KB {Sampled random image sizes in KB: [21,23,44,88,43,16,113,127,25,92], rounded avg to nearest 100}
* Engagement rate, made up of the sum of likes, replies, and retweets divided by the views, is equal to 0.05

In [18]:
total_tweets = 0
total_media_tweets = 0

total_id_characters = 0
total_created_chars = 0
total_lan_chars = 0
total_user_id_chars = 0
total_username_chars = 0

total_tweet_chars = 0
total_photos = 0
total_thumbnails = 0

for filename in filenames:
# filename = '/kaggle/input/elon-musk-tweets-2010-2021/2011.csv'
#     print('filename',filename)
    df = pd.read_csv(os.path.join(dirname, filename), usecols=['id','created_at','language','user_id_str','username','tweet','photos','video','thumbnail'])
    total_tweets += df.shape[0]
    
    total_id_characters += len(df.astype(str)['id'].sum())
    total_created_chars += len(df.astype(str)['created_at'].sum())
    total_lan_chars += len(df.astype(str)['language'].sum())
    total_user_id_chars += len(df.astype(str)['user_id_str'].sum())
    total_username_chars += len(df.astype(str)['username'].sum())
    
    total_tweet_chars += len(df.astype(str)['tweet'].sum())
    total_photos += df[df.astype(str)['photos'] != '[]']['photos'].count()
    total_thumbnails += df['thumbnail'].count()
#     print('thumbnail:',df['thumbnail'].count())
#     print('video',df['video'].sum())
#     print(total_photos)
#     print('photos:',df[df.astype(str)['photos'] != '[]']['photos'].count())

# print('id:',total_id_characters,'created at:',total_created_chars,'language:',total_lan_chars,'user_id:',total_user_id_chars,'username:',total_username_chars)
# print('tweet',total_tweet_chars)
total_chars = total_id_characters + total_created_chars + total_lan_chars + total_user_id_chars + total_username_chars + total_tweet_chars

total_videos = total_thumbnails - total_photos

character_bytes = total_chars * 1
photo_bytes = total_photos*100*1000
video_bytes = total_videos*25*1000000

total_bytes = character_bytes+photo_bytes+video_bytes
average_bytes = (int)(total_bytes/total_tweets)
average_total_data = average_bytes * average_views
print('Total data per tweet on average',average_total_data)

# print(total_tweets,total_bytes,total_videos,video_bytes,average_bytes)
# print('character',total_characters,'vids', total_videos, 'photos:',total_photos,'thumb',total_thumbnails)

# Q5. Estimate how many Elon Musk tweets are viewed per minute by the site's userbase
Assumption(s): 
* Engagement rate, made up of the sum of likes, replies, and retweets divided by the views, is equal to 0.05
* Includes all days of the 12 year time span (2010-2021)

In [19]:
engagement_rate = 0.05 # (likes + replies + retweets)/views
total_likes = 0
total_replies = 0
total_retweets = 0

for filename in filenames:
    df = pd.read_csv(os.path.join(dirname, filename), usecols=['nlikes', 'nreplies', 'nretweets'])
#     print("Year:", filename, "Number of Tweets in Year:",df.shape[0], "Number of Likes:", df.loc[:,'nlikes'].sum(), "Number of replies:",df.loc[:,'nreplies'].sum(), "Number of retweets:",df.loc[:,'nretweets'].sum())
    total_likes += df.loc[:,'nlikes'].sum()
    total_replies += df.loc[:,'nreplies'].sum()
    total_retweets += df.loc[:,'nretweets'].sum()

total_views = (int)((total_likes+total_replies+total_retweets)/engagement_rate)

total_years = 2021-2010+1
leap_years = 3
total_days = 365*(total_years-leap_years) + 366*(leap_years)
total_mins = total_days*1440

views_per_min = (int)(total_views/total_mins)
print('total minutes:',total_mins,'views per minute:',views_per_min)

# Q6. How much data is accessed per minute by Twitter in the process of displaying Elon Musk tweets
Assumption(s): 

In [20]:
print('data per tweet:', average_bytes,'bytes')
print('average tweet views per minute:',views_per_min)

data_per_min = average_bytes*views_per_min

print('data accessed per minute by twitter:', data_per_min,'bytes')

# Q7. How much data would be touched, on a monthly basis, by modifying a tweet

If one were to modify a tweet and update its 'likes' count, it would touch data. Earlier, it was calculated that the number total number of 'likes' is 633412855 and the total number of tweets is 43074. Over the course of 12 years, which is 144 months, Elon Musk is tweeting, on average, 300 times a month. He garners about 4.4 million likes per month, meaning this is the number of updates to the like counter every month. Assuming an integer is stored as a 4 byte value for each Tweet. This means there is 1200 bytes just for the 'like' variables. The update is occurring frequently, at about 100 likes per minute, and this data is shown to all users and accessed by about 2400 users every minute, assuming an engagement rate of 0.05 so displaying the updated like count to every user means around 400 million bytes, or 400 MB, is accessed and touched just for showing the like count to users on a monthly basis. So while the like counter isn't a large amount of data, just incrementing and storing the integers has a tremendous effect on how much users will interact with and access. 